In [40]:
import json
import random
import os

import pandas as pd

In [41]:
with open("data/dataset.json") as file:
    dataset = json.load(file)

run_name = "gpt-4o-2024-05-13_cotshot"
with open(f"output/{run_name}.json") as file:
    data = json.load(file)

for index, message in enumerate(dataset):
    try:
        if "sentiment" in data[index]:
            message["tools"][run_name] = data[index]['sentiment']
        else:
            message["tools"][run_name] = "invalid"
    except:
        message["tools"][run_name] = "invalid"

In [42]:
expected = [message["part2_aggregate"]["polarity"] if message["part2_aggregate"]["polarity"] != "undefined" else message["discussion_polarity"] for message in dataset]

actual = [x["tools"][run_name] for x in dataset]

In [92]:
disagreements = []

for index, _ in enumerate(expected):
    if expected[index] != actual[index]:
        disagreements.append(index)

disagreements_sample = random.sample(disagreements, round(len(disagreements)*0.30))

print(len(disagreements_sample))

114


In [93]:
csv_array = [["message", "expected", "actual"]]

for index in disagreements_sample:
    csv_array.append([dataset[index]["raw_message"], expected[index], actual[index]])

# Convert the data into a pandas DataFrame
df = pd.DataFrame(csv_array[1:], columns=csv_array[0])

# Save the DataFrame to an Excel spreadsheet
os.makedirs("output_sample", exist_ok=True)
df.to_excel(f"output_sample/{run_name}.xlsx", index=False)